# Titanic Predict survival

In [ ]:
import pandas as pd
import numpy as np

from pandas.core.common import random_state
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier


from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score

## Изучение и предобработка данных

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Обучающие данные

In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/projects/Predict_Titanic_survival/train.csv')

In [ ]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df_train.columns = df_train.columns.str.lower()

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   passengerid  891 non-null    int64  
 1   survived     891 non-null    int64  
 2   pclass       891 non-null    int64  
 3   name         891 non-null    object 
 4   sex          891 non-null    object 
 5   age          714 non-null    float64
 6   sibsp        891 non-null    int64  
 7   parch        891 non-null    int64  
 8   ticket       891 non-null    object 
 9   fare         891 non-null    float64
 10  cabin        204 non-null    object 
 11  embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [ ]:
df_train['sibsp_parch'] = df_train['parch'] + df_train['sibsp']

In [ ]:
def family_size(row):
  if row == 0:
    return 0
  elif row >= 3:
    return 1
  else:
    return 2

In [ ]:
df_train['family_size'] = df_train['sibsp_parch'].apply(family_size)

In [ ]:
df_train['level'] = df_train['cabin'].astype(str).str[0]

In [ ]:
df_train['level'].value_counts()

n    687
C     59
B     47
D     33
E     32
A     15
F     13
G      4
T      1
Name: level, dtype: int64

In [ ]:
df_train['level'] = df_train['level'].replace(
    {'n': 0, 'C': 1, 'B': 2, 'D': 3, 'E': 4, 'A': 5, 'F': 6, 'G': 7, 'T': 8}
     )

In [ ]:
df_train['sex'] = df_train['sex'].replace({'male': 0, 'female': 1})

In [ ]:
df_train['sex'].value_counts()

0    577
1    314
Name: sex, dtype: int64

In [ ]:
df_train['sex'] = df_train['sex'].astype('int')

In [ ]:
df_train['age'] = df_train['age'].fillna(df_train['age'].mean())

In [ ]:
df_train['age'] = df_train['age'].astype('int')

In [ ]:
df_train['embarked'].value_counts()

S    644
C    168
Q     77
Name: embarked, dtype: int64

In [ ]:
df_train['embarked'] = df_train['embarked'].replace({'S': 0, 'C': 1, 'Q': 2})

In [ ]:
df_train['embarked'] = df_train['embarked'].fillna(2)

In [ ]:
df_train['embarked'] = df_train['embarked'].astype('int')

In [ ]:
df_train = df_train[['survived', 'pclass','sex', 'age', 'family_size', 'fare', 'embarked', 'level']]

In [ ]:
df_train.head()

,survived,pclass,sex,age,family_size,fare,embarked,level
0,0,3,0,22,2,7.2500,0,0
1,1,1,1,38,2,71.2833,1,1
2,1,3,1,26,0,7.9250,0,0
3,1,1,1,35,2,53.1000,0,1
4,0,3,0,35,0,8.0500,0,0


### Тестовые данные

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/projects/Predict_Titanic_survival/test.csv')

In [ ]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
df_test.columns = df_test.columns.str.lower()

In [ ]:
df_test_passengerid = df_test['passengerid']

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   passengerid  418 non-null    int64  
 1   pclass       418 non-null    int64  
 2   name         418 non-null    object 
 3   sex          418 non-null    object 
 4   age          332 non-null    float64
 5   sibsp        418 non-null    int64  
 6   parch        418 non-null    int64  
 7   ticket       418 non-null    object 
 8   fare         417 non-null    float64
 9   cabin        91 non-null     object 
 10  embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [ ]:
df_test['sibsp_parch'] = df_test['parch'] + df_test['sibsp']

In [ ]:
def family_size(row):
  if row == 0:
    return 0
  elif row >= 3:
    return 1
  else:
    return 2

In [ ]:
df_test['family_size'] = df_test['sibsp_parch'].apply(family_size)

In [ ]:
df_test['level'] = df_test['cabin'].astype(str).str[0]

In [ ]:
df_test['level'] = df_test['level'].replace(
    {'n': 0, 'C': 1, 'B': 2, 'D': 3, 'E': 4, 'A': 5, 'F': 6, 'G': 7, 'T': 8}
     )

In [ ]:
df_test['sex'] = df_test['sex'].replace({'male': 0, 'female': 1})

In [ ]:
df_test['sex'].value_counts()

0    266
1    152
Name: sex, dtype: int64

In [ ]:
df_test['sex'] = df_test['sex'].astype('int')

In [ ]:
df_test['age'] = df_test['age'].fillna(df_test['age'].mean())

In [ ]:
df_test['age'] = df_test['age'].astype('int')

In [ ]:
df_test['embarked'] = df_test['embarked'].replace({'S': 0, 'C': 1, 'Q': 2})

In [ ]:
df_test['embarked'] = df_test['embarked'].fillna(2)

In [ ]:
df_test['embarked'] = df_test['embarked'].astype('int')

In [ ]:
df_test['fare'] = df_test['fare'].fillna(df_test['fare'].mean())

In [ ]:
df_test = df_test[['pclass','sex', 'age', 'family_size', 'fare', 'embarked', 'level']]

In [ ]:
df_test.head()

,pclass,sex,age,family_size,fare,embarked,level
0,3,0,34,0,7.8292,2,0
1,3,1,47,2,7.0000,0,0
2,2,0,62,0,9.6875,2,0
3,3,0,27,0,8.6625,0,0
4,3,1,22,2,12.2875,0,0


## Исследование моделей

In [ ]:
df_train['survived'].mean()

0.3838383838383838

In [ ]:
features_train = df_train.drop('survived', axis=1)
target_train = df_train['survived']

In [ ]:
features_train, features_valid = train_test_split(features_train, test_size=0.25, random_state=12345)
target_train, target_valid = train_test_split(target_train, test_size=0.25, random_state=12345)

In [ ]:
df_volumes = pd.DataFrame(
    {
     "Примечание": ['Доля обучающих признаков', 'Кол. обущаюших признаков', 'Доля целевых признаков', 'Кол. целевых признаков'],   
     "Обущающая_выборка": [round(len(features_train) / len(df_train), 2), len(features_train), round(len(target_train) / len(df_train), 2), len(target_train),],
    }
)
df_volumes

,Примечание,Обущающая_выборка
0,Доля обучающих признаков,0.75
1,Кол. обущаюших признаков,668.00
2,Доля целевых признаков,0.75
3,Кол. целевых признаков,668.00


In [ ]:
%%time


rf = RandomForestClassifier(random_state=123)

params = {'n_estimators': [350, 400],
          'max_depth': [3, 7, 10],
          'criterion':['gini'],
          'min_samples_leaf' : [1, 3, 5],
          'max_features':['auto'],
          'min_samples_split': [5, 10],
          'max_leaf_nodes':[3, 5],
          }

cv_rfc_model = GridSearchCV(estimator=rf, param_grid=params, n_jobs=-1, cv=10)

cv_rfc_model.fit(features_train, target_train)

CPU times: user 7.33 s, sys: 431 ms, total: 7.76 s
Wall time: 6min 44s


In [ ]:
cv_rfc_model.best_params_
cv_rfc_model.best_score_

0.8187697874265039

In [ ]:
rfc_best_model = RandomForestClassifier(max_depth=7, max_leaf_nodes=5, min_samples_leaf=1, min_samples_split=5, n_estimators=350, random_state=123)
rfc_best_model.fit(features_train, target_train)

RandomForestClassifier(max_depth=7, max_leaf_nodes=5, min_samples_split=5,
                       n_estimators=350, random_state=123)

In [ ]:
rfc_best_model.score(features_valid, target_valid)

0.7668161434977578

In [ ]:
%%time 

rfc = rfc_best_model.predict(df_test)
df_test['survived'] = list(rfc)

CPU times: user 70.4 ms, sys: 0 ns, total: 70.4 ms
Wall time: 71.5 ms


In [ ]:
df_test['passengerid'] = df_test_passengerid
df_submission = df_test[['passengerid', 'survived']]
df_submission.tail()

,passengerid,survived
413,1305,0
414,1306,1
415,1307,0
416,1308,0
417,1309,0


In [ ]:
# df_submission.to_csv('/content/drive/MyDrive/df_submission1.csv', index=False)

In [ ]:
## 0.67942 - 1 заход
## 0.77272 - 2 заход
## 0.77990 - 3 заход